## Projecte 10 - Optimització de rutes per a vehicles autònoms de repartiment

Integrants:
- **Nom:** David Morillo, **NIU:** 1666540
- **Nom:** Adrià Muro, **NIU:** 1665191
- **Nom:** Lucia Garrido, **NIU:** 1671463
- **Nom:** Albert Guillaumet, **NIU:** 1672344

## Introducció

En aquest projecte, abordarem el problema de l'optimització de rutes per a vehicles autònoms de repartiment. L'objectiu principal és minimitzar el cost total de les rutes que han de seguir els vehicles per tal de satisfer la demanda de tots els clients. Aquest tipus de problema és conegut com a Vehicle Routing Problem (VRP) i és fonamental en la logística i la gestió de la cadena de subministrament.

El VRP es pot descriure com un problema d'assignació de rutes a una flota de vehicles que han de visitar un conjunt de clients amb demandes específiques. Cada vehicle té una capacitat limitada, i el cost de viatjar entre els clients pot variar. A més, cada client ha de ser atès exactament una vegada, i cada vehicle ha de començar i acabar la seva ruta al dipòsit.

Per resoldre aquest problema, utilitzarem tècniques d'optimització matemàtica i programació lineal. Definirem variables de decisió que ens permetran modelar les rutes dels vehicles i les assignacions dels clients. També formularem una funció objectiu que minimitzi el cost total de les rutes, així com un conjunt de restriccions que assegurin que totes les demandes dels clients es compleixen i que no es superen les capacitats dels vehicles.

Aquest projecte és una aplicació pràctica de tècniques avançades d'optimització i pot tenir un impacte significatiu en la millora de l'eficiència operativa de les empreses de repartiment.

### Definir Variables

- `N`: Nombre de clients.
- `K`: Nombre de vehicles.
- `Q`: Capacitat de cada vehicle.
- $d_i$: Demanda del client $i$.
- $c_{ij}$: Cost de viatjar del client $i$ al client $j$.
- $x_{ij}$: Variable binària que indica si hi ha una ruta del client $i$ al client $j$.
- $y_i$: Variable binària que indica si el client $i$ és atès.

### Funció Objectiu

- Minimitzar el cost total de les rutes:
  
  $$
  \text{Minimitzar} \sum_{i=0}^{N} \sum_{j=0}^{N} c_{ij} \cdot x_{ij}
  $$

### Restriccions

- Cada client ha de ser atès exactament una vegada:
  
  $$
  \sum_{j=1, j \neq i}^{N} x_{ij} = y_i
  $$

  $$
  \sum_{i=1, i \neq j}^{N} x_{ij} = y_j
  $$

- La suma de les demandes dels clients en una ruta no ha de superar la capacitat del vehicle:
  
  $$
  \sum_{i=1}^{N} d_i \cdot y_i \leq Q
  $$

- Cada vehicle ha de començar i acabar al dipòsit:
  
  $$
  \sum_{j=1}^{N} x_{0j} = K
  $$
  
  $$
  \sum_{i=1}^{N} x_{i0} = K
  $$

# Implementació

In [66]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.


# Exemple d'aplicació

In [67]:
def generate_random_matrix(n):
    import random
    matrix = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                matrix[i][j] = random.randint(1, 10)
    return matrix

dist = generate_random_matrix(10)
dist

[[0, 5, 5, 1, 9, 7, 6, 7, 1, 9],
 [5, 0, 9, 4, 9, 5, 8, 4, 10, 4],
 [7, 1, 0, 4, 7, 2, 5, 5, 9, 3],
 [2, 6, 10, 0, 6, 2, 8, 7, 5, 8],
 [10, 1, 5, 9, 0, 7, 5, 10, 4, 7],
 [1, 10, 7, 6, 7, 0, 1, 5, 8, 9],
 [5, 3, 2, 1, 5, 2, 0, 4, 7, 9],
 [4, 7, 2, 7, 8, 1, 5, 0, 4, 10],
 [4, 7, 3, 9, 7, 4, 4, 4, 0, 8],
 [8, 7, 6, 4, 5, 5, 1, 2, 6, 0]]

In [68]:
import pulp

# Definir el problema
vrp = pulp.LpProblem("Vehicle_Routing_Problem", pulp.LpMinimize)

# Paràmetres
N = 10  # Número de clients
K = 3   # Número de vehicles
Q = 15  # Capacitat de cada vehicle
demanda = [2, 4, 3, 5, 2, 7, 3, 4, 6, 5]  # Demanda de cada client (sense incloure dipòsit)
demanda = [0] + demanda  # Incloent el dipòsit com a client 0
costos = generate_random_matrix(N + 1)  # Costos de viatge entre clients (incloent dipòsit)

# Variables de decisió
x = pulp.LpVariable.dicts(
    "x", 
    ((i, j, k) for i in range(N + 1) for j in range(N + 1) for k in range(K)), 
    cat='Binary'
)
y = pulp.LpVariable.dicts(
    "y", 
    ((i, k) for i in range(N + 1) for k in range(K)), 
    cat='Binary'
)

# Funció objectiu: minimitzar el cost total de viatge
vrp += pulp.lpSum(costos[i][j] * x[i, j, k] for i in range(N + 1) for j in range(N + 1) for k in range(K))

# Restriccions

# Cada client ha de ser atès exactament una vegada
for i in range(1, N + 1):
    vrp += pulp.lpSum(y[i, k] for k in range(K)) == 1, f"Atendre_client_{i}"

# Capacitat dels vehicles
for k in range(K):
    vrp += pulp.lpSum(demanda[i] * y[i, k] for i in range(1, N + 1)) <= Q, f"Capacitat_vehicle_{k}"

# Cada vehicle ha de sortir i tornar al dipòsit exactament una vegada
for k in range(K):
    vrp += pulp.lpSum(x[0, j, k] for j in range(1, N + 1)) == 1, f"Sortida_dipòsit_{k}"
    vrp += pulp.lpSum(x[i, 0, k] for i in range(1, N + 1)) == 1, f"Tornada_dipòsit_{k}"

# Flux d'entrada i sortida de cada client
for k in range(K):
    for i in range(1, N + 1):
        vrp += pulp.lpSum(x[i, j, k] for j in range(N + 1) if j != i) == y[i, k], f"Flux_sortida_{i}_vehicle_{k}"
        vrp += pulp.lpSum(x[j, i, k] for j in range(N + 1) if j != i) == y[i, k], f"Flux_entrada_{i}_vehicle_{k}"

# Prohibir bucles (un client no pot visitar-se a si mateix)
for i in range(N + 1):
    for k in range(K):
        vrp += x[i, i, k] == 0, f"Prohibir_bucle_{i}_vehicle_{k}"

# Resoldre el problema
vrp.solve()

# Imprimir resultats
print(f"Estat de la solució: {pulp.LpStatus[vrp.status]}")
print(f"Cost total: {pulp.value(vrp.objective)}")

# Crear matriu de rutes
matriu_rutes = [[0 for _ in range(N + 1)] for _ in range(N + 1)]

# Omplir la matriu amb els valors de les variables de decisió
for i in range(N + 1):
    for j in range(N + 1):
        for k in range(K):
            if pulp.value(x[i, j, k]) == 1:
                matriu_rutes[i][j] = 1

# Imprimir la matriu de rutes
print("\nMatriu de rutes:")
for fila in matriu_rutes:
    print(fila)


Estat de la solució: Optimal
Cost total: 29.0

Matriu de rutes:
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


# Aplicació real

Dataset: https://www.kaggle.com/datasets/sujalsuthar/amazon-delivery-dataset

In [69]:
datafile = "data/amazon_delivery.csv"

import pandas as pd
import numpy as np


df = pd.read_csv(datafile).head(10)

distance_matrix = np.array([
    [np.sqrt((row['Store_Latitude'] - row2['Drop_Latitude'])**2 + 
             (row['Store_Longitude'] - row2['Drop_Longitude'])**2)
     for _, row2 in df.iterrows()]
    for _, row in df.iterrows()
])

# posar diagonal a 0
np.fill_diagonal(distance_matrix, 0)

# Imprimir la matriu de distàncies
print("\nMatriu de distàncies:")
for fila in distance_matrix:
    print(["{:.2f}".format(dist) for dist in fila])
    
df.head(10)


Matriu de distàncies:
['0.00', '9.89', '9.98', '11.75', '10.68', '5.86', '9.59', '10.30', '5.81', '7.97']
['10.01', '0.00', '0.01', '1.97', '2.61', '4.61', '13.11', '1.04', '4.72', '17.49']
['10.01', '0.19', '0.00', '1.97', '2.61', '4.61', '13.12', '1.04', '4.72', '17.49']
['11.81', '2.20', '2.05', '0.00', '3.88', '6.62', '15.08', '1.50', '6.74', '19.43']
['10.71', '2.44', '2.56', '3.75', '0.00', '4.84', '11.72', '3.55', '4.91', '17.56']
['5.89', '4.43', '4.56', '6.53', '4.80', '0.00', '9.28', '5.22', '0.17', '12.97']
['9.45', '12.78', '12.95', '14.86', '11.52', '9.08', '0.00', '13.88', '8.96', '10.08']
['10.44', '1.39', '1.22', '1.36', '3.74', '5.43', '14.21', '0.00', '5.55', '18.11']
['5.88', '4.43', '4.56', '6.52', '4.81', '0.06', '9.30', '5.22', '0.00', '12.97']
['7.86', '17.29', '17.41', '19.30', '17.46', '12.87', '10.09', '17.89', '12.77', '0.00']


,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys
5,fxuu788413734,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,Urban,130,Toys
6,njmo150975311,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,19:30:00,Fog,Jam,scooter,Metropolitian,200,Toys
7,jvjc772545076,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,17:30:00,Cloudy,Medium,motorcycle,Metropolitian,160,Snacks
8,uaeb808891380,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,21:05:00,Stormy,Jam,motorcycle,Metropolitian,170,Electronics
9,bgvc052754213,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,22:10:00,Fog,Jam,motorcycle,Metropolitian,230,Toys


In [ ]:
import pulp

# Definir el problema
vrp = pulp.LpProblem("Vehicle_Routing_Problem", pulp.LpMinimize)

# Paràmetres
N = distance_matrix.shape[0]  # Número de clients
K = 3   # Número de vehicles
Q = 15  # Capacitat de cada vehicle
demanda = np.random.randint(1, 10, size=N+1)  # Demanda aleatòria per a cada client
demanda = [0] + demanda  # Incloent el dipòsit com a client 0
costos = np.zeros((distance_matrix.shape[0] + 1, distance_matrix.shape[1] + 1))
costos[1:, 1:] = distance_matrix  # Costos de viatge entre clients (incloent dipòsit)

# Variables de decisió
x = pulp.LpVariable.dicts(
    "x", 
    ((i, j, k) for i in range(N + 1) for j in range(N + 1) for k in range(K)), 
    cat='Binary'
)
y = pulp.LpVariable.dicts(
    "y", 
    ((i, k) for i in range(N + 1) for k in range(K)), 
    cat='Binary'
)

# Funció objectiu: minimitzar el cost total de viatge
vrp += pulp.lpSum(costos[i][j] * x[i, j, k] for i in range(N + 1) for j in range(N + 1) for k in range(K))

# Restriccions

# Cada client ha de ser atès exactament una vegada
for i in range(1, N + 1):
    vrp += pulp.lpSum(y[i, k] for k in range(K)) == 1, f"Atendre_client_{i}"

# Capacitat dels vehicles
for k in range(K):
    vrp += pulp.lpSum(demanda[i] * y[i, k] for i in range(1, N + 1)) <= Q, f"Capacitat_vehicle_{k}"

# Cada vehicle ha de sortir i tornar al dipòsit exactament una vegada
for k in range(K):
    vrp += pulp.lpSum(x[0, j, k] for j in range(1, N + 1)) == 1, f"Sortida_dipòsit_{k}"
    vrp += pulp.lpSum(x[i, 0, k] for i in range(1, N + 1)) == 1, f"Tornada_dipòsit_{k}"

# Flux d'entrada i sortida de cada client
for k in range(K):
    for i in range(1, N + 1):
        vrp += pulp.lpSum(x[i, j, k] for j in range(N + 1) if j != i) == y[i, k], f"Flux_sortida_{i}_vehicle_{k}"
        vrp += pulp.lpSum(x[j, i, k] for j in range(N + 1) if j != i) == y[i, k], f"Flux_entrada_{i}_vehicle_{k}"

# Prohibir bucles (un client no pot visitar-se a si mateix)
for i in range(N + 1):
    for k in range(K):
        vrp += x[i, i, k] == 0, f"Prohibir_bucle_{i}_vehicle_{k}"

# Resoldre el problema
vrp.solve()

# Imprimir resultats
print(f"Estat de la solució: {pulp.LpStatus[vrp.status]}")
print(f"Cost total: {pulp.value(vrp.objective):.3f}")

# Crear matriu de rutes
matriu_rutes = [[0 for _ in range(N + 1)] for _ in range(N + 1)]

# Omplir la matriu amb els valors de les variables de decisió 
for i in range(N + 1):
    for j in range(N + 1):
        for k in range(K):
            if pulp.value(x[i, j, k]) == 1:
                matriu_rutes[i][j] = 1

# Imprimir la matriu de rutes
print("\nMatriu de rutes:")
for fila in matriu_rutes:
    print(fila)
    
# veure rutes per vehicle
print("")
for k in range(K):
    print(f"Ruta del vehicle {k}:")
    ruta = [i for i in range(N + 1) if pulp.value(y[i, k]) == 1]
    print(ruta)


Estat de la solució: Optimal
Cost total: 9.215

Matriu de rutes:
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Ruta del vehicle 0:
[4, 5, 8, 10]
Ruta del vehicle 1:
[1, 2, 3]
Ruta del vehicle 2:
[6, 7, 9]
